In [74]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer

#dlsite
#fkey作者來源key.txt
#fcomiclist結果key_dlsite.text

#輸出格式：
#dlsite
#!作者
#!總筆數
#==類別_數量_中文敘述
#類別_品名
#網址

#頁顯示數量
pnum = 30
mlink = 'http://www.dlsite.com/books/'

fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()
key='駄菓子'

#key=作者
key2 = urllib.quote(key.decode('utf8').encode('euc_jp'))
key3 = urllib.unquote(key2.decode('euc_jp').encode('utf8'))
#網址用
key4 = urllib.quote(key)
#print key4, '\n' , urllib.quote(key.decode('sjis').encode('utf8')) 

#檢查BOM
if '%EF%BB%BF' in urllib.quote(key):
    print 'fuck ms'

'''網址樣本
http://www.dlsite.com/maniax/fsr/=/language/jp/sex_category%5B0%5D/male/keyword/
urllib.quote(key)
/ana_flg/all/order%5B0%5D/release_d/genre_and_or/or/options_and_or/or/per_page/30/show_type/n/page/
1
'''
link = "http://www.dlsite.com/maniax/fsr/=/language/jp/sex_category%5B0%5D/male/keyword/"+urllib.quote(key)+"/ana_flg/all/order%5B0%5D/release_d/genre_and_or/or/options_and_or/or/per_page/30/show_type/n/page/1"

#head = {'User-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36'}
res = requests.get(link)#, headers = head)

res.encoding =  res.apparent_encoding#亂碼處理
soup = BeautifulSoup(res.text,"html.parser")#超出lxml緩存，改其他存取

def next(page = 2):
    link = "http://www.dlsite.com/maniax/fsr/=/language/jp/sex_category%5B0%5D/male/keyword/"+urllib.quote(key)+"/ana_flg/all/order%5B0%5D/release_d/genre_and_or/or/options_and_or/or/per_page/30/show_type/n/page/"+str(page)

    
    res = requests.get(link)
    res.encoding =  res.apparent_encoding
    
    only_a_tags = SoupStrainer(id="search_result_list")#縮小處理範圍
    #先html.parser解析與縮小範圍，再以字串給lxml
    #soup = BeautifulSoup(str(BeautifulSoup(res.text,"html.parser",  parse_only=only_a_tags)),"lxml")
    soup = BeautifulSoup(res.text,"html.parser",  parse_only=only_a_tags)
    
    return soup

#全轉半_含轉小寫
def Q2B(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    ustring=ustring.lower()
    return ustring
#半轉全_含轉小寫
def B2Q(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    ustring=ustring.lower()
    for tm in ustring:
        if hs.find(tm)+1:
            tm = fs[hs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    return ustring

#資料儲存
def save(sdict , check=0):
    #print listdata
    for temp in listdata:
        if sdict.get(temp):
            #fout.write(temp.encode('utf8') +sdict[temp].encode('utf8')  + '\n')
            fout.write(sdict[temp].encode('utf8')  + '\n')
    #return

#作品頁面
def bfind(blink='http://www.dlsite.com/books/'):
    res = requests.get(blink)
    res.encoding =  res.apparent_encoding
    ssoup_1 = SoupStrainer(id="work_right")#資訊欄位
    ssoup_2 = SoupStrainer(class_="work_article work_story")#內容欄位
    
    #only_a_tags = SoupStrainer(class_="work_article work_story")#縮小處理範圍
    #only_a_tags = SoupStrainer(['a','td'])#縮小處理範圍
    #先html.parser解析與縮小範圍，再以字串給lxml
    #soup = BeautifulSoup(str(BeautifulSoup(res.text,"html.parser",  parse_only=only_a_tags)),"lxml")
    soup1 = BeautifulSoup(res.text,"lxml",  parse_only=ssoup_1)#資訊欄位
    soup2=BeautifulSoup(res.text,"lxml",  parse_only=ssoup_2)#內容欄位
    #print soup.text
    #soup = BeautifulSoup(res.text, 'lxml', parse_only=SoupStrainer(["main_genre" , class_="work_genre"]))
    nnum1=''#作者出現次數
    nnum2=''#作者出現次數
    bdate1=''#含作者資料
    bdate2=''#含作者資料
    check=0
    cname=''
    
    nnum1 = Q2B(soup1.text).encode('utf8').count(key)#作者出現次數
    nnum2=Q2B(soup2.text).encode('utf8').count(key)
    
    #資訊欄位
    for tm in soup1.select('tr'):
        #print tm
        #日期
        if u'予告開始日' in tm.text:
            #print 'ooooooooooooooooooooooooook',tm.td
            check=1
            cdata=tm.td.text
        elif u'販売日' in tm.text:
            cdata=tm.td.text
        #類型
        if u'作品形式'in tm.text:
            ctype='['+tm.td.text+']'
        #作者
        ##作者不一定有<a>，棄用的方法。
        #tmm=tm.select('a')#.get_text('//')
        tmm=tm.td
        tm=Q2B(tm.text).replace('\n','')
        #tm=Q2B(tm.get_text())
        if key.decode('utf8') in tm:
            
            #for tmmm in tmm:
                #cname=cname+tmmm.text+u'/'
            #print tmm,len(tmm)
            cname=Q2B(tmm.text).replace('\n','')
            bdate1=bdate1+tm+'_'
    bdate1=bdate1[:-1]
    #cname=Q2B(cname[:-1])
    #print 'cname:',len(cname),cname
    
    #內容欄位
    x=0
    if nnum2 > 0:
        tm=Q2B(soup2.text)
        while tm.find(key.decode('utf8'))>-1:
            #print 's2',tm[tm.rfind('\n',0,tm.find(key))+1:tm.find('\n',tm.find(key))]
            #key前後\n
            bdate2=bdate2+'-'+tm[tm.rfind(u'\n',0,tm.find(key.decode('utf8')))+1 : tm.find(u'\n',tm.find(key.decode('utf8')))].replace(u'\n','')+'\n'
            #print tm.find(key),tm.rfind('\n',0,tm.find(key)),tm.find('\n',tm.find(key))
            tm=tm[tm.find(key.decode('utf8'))+1:]
            x=x+1
            if x>5:
                break
    #\n佔用兩格
    bdate2=bdate2[:-2]
    #sou = BeautifulSoup(str(tr),"lxml")
    #'''
    
    if int(nnum1) > 0:
        check=5
        if u'マンガ' in ctype:
            check=2
            if u'同人' in ctype:
                check=4
            elif key.decode('utf8') != cname:
                check=3
                
    else:
        if u'マンガ' in ctype:
            check=6
        else:
            check=7
    
    
    
    '''
    1新
    2漫單
    3漫多
    4漫同人
    5它
    6漫
    7它
    1>4>3>2
    
    新
    合1:漫_漫多_同_它
    合2:漫_非
    #'''
    
    #print 'bfind'
    
    #類型_nnum1_bdate1_nnum2_bdate2_check
    #return cdata,ctype,nnum1,bdate1,nnum2,bdate2
    return ctype,nnum1,bdate1,nnum2,bdate2,check

#資料處理
def findbook(soup , page = 1):
    a =0
    check = 0
    #print len(soup.select('.work_thumb'))
    
    #print soup#.select('tr')[:2]
    for work_thumb in soup.select('.work_thumb'):
        #print tr.text
        if a==10:
            #sys.exit()################
            print '10'
        
        tr = soup.select('tr')[3*a]
        #print tr.text
        sou = BeautifulSoup(str(tr),"lxml")
        #print sou.text
        
        ctype = sou.select('.work_genre')[0].text
        cbook = sou.select('.work_name')[0].text
        cdata = sou.select('.sales_date')[0].text#[5:]#next_sibling.next_sibling[5:]
        #print ctype,cbook,cdata
        #print cdata,'\n',ctype
        
        
        
        #網址
        blink = ''
        blink = sou.select('.work_name')[0].select('a')[0].get('href')
        #blink = mlink# + blink[3:]
        #print blink,cbook
        
        #作品處理
        #日期_類型_nnum1_bdate1_nnum2_bdate2
        cfind=bfind(blink)
        #print '########catch:',len(cfind)#,cfind[:]
        #cdata=cfind[0]
        ctype=cfind[0]
        cnum1=str(cfind[1]).decode('utf8')
        cdate1=cfind[2]
        cnum2=str(cfind[3]).decode('utf8')
        cdate2=cfind[4]
        check=cfind[5]
        #print 'check:',check
        #print type(ctype),type(str(cnum1)),type(cdate1),type(cnum2),type(cdate2)
        
        #品名
        book = cbook.replace('\n','')
        
        dtype=ctype
        
        #日期處理
        #予告開始日 : 	2015年04月24日
        #販売日: 2015年08月21日
        if u'予告開始日' in cdata:
            cdata=cdata[7:].replace(u'年','').replace(u'月','').replace(u'日','')
            check=1
        else:
            cdata=cdata[5:].replace(u'年','').replace(u'月','').replace(u'日','')
        
        
        #cdata = cdata.rstrip()
        if len(cdata) < 4:#無日期
            cdata = u'0000/00/00'#填入日期
            #check = 4#新作
        data = cdata
        while listdata.count(data):#重複日期判斷
            data = data[:6] + str(int(data[6:]) + 1).rjust(2,'0')#日期+1_十位數填0
        listdata.append(data)
        #print data,data[:6]
        #'''
        
        '''
        1新
        日_類_作_
        !nnum1!網
        2漫單
        類_作_
        3漫多
        類_作_
        4漫同人
        類_作_
        !網
        5它
        類_作_
        !nnum1!網
        6漫
        7它
        #'''
        
        #寫入dict
        #book = dtype + '_' + book + '_' + '\n!' + blink#類形+書名+網址
        #print book
        if check == 1:
            book=data+'_'+dtype+'_'+book+'_\n!'+cnum1+'!'+blink
            dict1.setdefault(data,book)
        elif check == 2:
            book=dtype+'_'+book+'_'+cdate1+'_'
            dict2.setdefault(data,book)
        elif check == 3:
            book=dtype+'_'+book+'_'+cdate1+'_'
            dict3.setdefault(data,book)
        elif check == 4:
            book=dtype+'_'+book+'_'+cdate1+'_'
            dict4.setdefault(data,book)
        elif check == 5:
            book=dtype+'_'+book+'_'+cdate1+'_\n!'+cnum1+'!'+cnum2+'!'+blink
            dict5.setdefault(data,book)
        elif check == 6:
            book=dtype+'_'+book+'_\n'+cdate2+'_\n!'+cnum1+'!'+cnum2+'!'+blink
            dict6.setdefault(data,book)
        else:
            book=dtype+'_'+book+'_\n'+cdate2+'_\n!'+cnum1+'!'+cnum2+'!'+blink
            dict7.setdefault(data,book)
        print check,'_',book
        #'''
        a = a + 1
    #print '========'
    #return

########
key=key.lower()

pn=''
#print soup.select('strong')
if len(soup.select('strong'))>0:
    pn = soup.select('strong')[0].text#資料筆數
    print type(pn)
#else: 
    

print pn

#資料筆數_是否數字
if pn.isdigit():
    #print soup.find_all('b')
    
    if int(pn) > pnum:
        print 'BIG'
    
    fout = open(key.decode('utf8') + '_dlsite.txt', 'w')#寫入模式開檔
    fout.write('dlsite\n')#getchu
    print key.decode('utf8') , pn , 'num\n========v1'
    time.sleep(1)
    fout.write('!' + key + '\n!總筆數' + pn.encode('utf8') + '\n')
    
    p = 0#頁
    #建空輸出用字典與陣列
    dict1={}#新
    dict2={}#漫單
    dict3={}#漫多
    dict4={}#漫同人
    dict5={}#它
    dict7={}#漫
    dict6={}#它
    listdata = []
    check=0
    
    #資料處理
    while (int(pn) - p * pnum) > 0:
        p = p + 1
        print 'page:' + str(p)
        soup = next(p)#頁
        
        findbook(soup)#資料處理
        time.sleep(1)
        
    #print 'ook\n',listdata
    
    #日期排序
    listdata.sort()
    
    temp = ''
    #dict1_新輸出
    fout.write('==new_' + str(len(dict1)) +'_新\n')
    save(dict1)
    #dict2_漫單輸出
    fout.write('==book_' + str(len(dict2)) +'_漫單\n')
    save(dict2)
    #dict3_漫多輸出
    fout.write('==numbook_' + str(len(dict3)) +'_漫多\n')
    save(dict3)
    #dict4_漫同人輸出
    fout.write('==dbook_' + str(len(dict4)) +'_漫同人\n')
    save(dict4)
    #dict5_它輸出
    fout.write('==other_' + str(len(dict5)) +'_它\n')
    save(dict5)
    #dict6_漫輸出
    fout.write('==obook_' + str(len(dict6)) +'_漫\n')
    save(dict6)
    #dict7_它輸出
    fout.write('==oother_' + str(len(dict7)) +'_它\n')
    save(dict7)
    
    fout.close()
    print 'ok'
elif pn:
    print 'No Date'

#結束讀秒
x=3
while x!=0:
    print x,'..',
    x=x-1
    time.sleep(1)
print 'end'


<type 'unicode'>
69
BIG
駄菓子 69 num
========v1
page:1
6 _ [マンガ]_COMIC真激2015年5月号_
-駄菓子の利益は薄いんだぞ!!
-山崎かな『駄菓子菓子lw』
-誘惑に負けて売り物の駄菓子を食べ漁る!! 
-駄菓子の代金は? →子供銀行券 百万円でお支払い!! →_
!0!4!http://www.dlsite.com/books/work/=/product_id/BJ060865.html
7 _ [同人ソフトその他同人ソフト]_へそ責め ～3人の食料～_
-cg:駄菓子_
!0!1!http://www.dlsite.com/maniax/work/=/product_id/RJ162644.html
7 _ [同人ソフトイラスト集(CG集)]_僕らは透明人間の餌食_
-4:駄菓子屋の前のショタに強制フェラ_
!0!1!http://www.dlsite.com/maniax/work/=/product_id/RJ161156.html
3 _ [マンガ]_COMIC快楽天ビースト 2015年 8月号_著者 : redrop / 智弘カイ / 駄菓子 / 雛咲葉 / いがらし游真 / いちこ / 安部マナブ / cuvie / 夏桜 / 悠木しん_
5 _ [同人ソフトデジタルコミック]_駄菓子解剖図鑑5ドッキリ玩具編_シリーズ名 : 駄菓子解剖図鑑_
!1!1!http://www.dlsite.com/home/work/=/product_id/RJ158857.html
7 _ [同人ソフトその他同人ソフト]_腹パン ～三人の生贄～_
-イラスト:駄菓子屋_
!0!1!http://www.dlsite.com/maniax/work/=/product_id/RJ159210.html
3 _ [マンガ]_COMIC快楽天ビースト 2015年 6月号_著者 : みくに瑞貴 / ぴょん吉 / いがらし游真 / 雛咲葉 / kakao / さんろく丸 / 駄菓子 / 武将武 / 夏桜 / 赤城あさひと_
7 _ [アドベンチャーゲーム / 7人の幼馴染からエッチに尽くされちゃう学園アドベンチャー]_ずっとつくしてあげるの！_
-駄菓子屋を営んでいる祖母を手伝いながら、二人で暮らしている。_


In [69]:
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer

#頁顯示數量
pnum = 30
mlink = 'http://www.dlsite.com/books/'

fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()
def Q2B(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    ustring=ustring.lower()
    return ustring

#def bfind(blink='http://www.dlsite.com/books/work/=/product_id/BJ041839.html'):
#def bfind(blink='http://www.dlsite.com/pro/work/=/product_id/VJ009052.html'):
def bfind(blink='http://www.dlsite.com/books/'):
    res = requests.get(blink)
    res.encoding =  res.apparent_encoding
    ssoup_1 = SoupStrainer(id="work_right")#資訊欄位
    ssoup_2 = SoupStrainer(class_="work_article work_story")#內容欄位
    
    #only_a_tags = SoupStrainer(class_="work_article work_story")#縮小處理範圍
    #only_a_tags = SoupStrainer(['a','td'])#縮小處理範圍
    #先html.parser解析與縮小範圍，再以字串給lxml
    #soup = BeautifulSoup(str(BeautifulSoup(res.text,"html.parser",  parse_only=only_a_tags)),"lxml")
    soup1 = BeautifulSoup(res.text,"lxml",  parse_only=ssoup_1)#資訊欄位
    soup2=BeautifulSoup(res.text,"lxml",  parse_only=ssoup_2)#內容欄位
    #print soup.text
    #soup = BeautifulSoup(res.text, 'lxml', parse_only=SoupStrainer(["main_genre" , class_="work_genre"]))
    nnum1=''#作者出現次數
    nnum2=''#作者出現次數
    bdate1=''#含作者資料
    bdate2=''#含作者資料
    check=0
    cname=''
    
    nnum1 = Q2B(soup1.text).encode('utf8').count(key)#作者出現次數
    nnum2=Q2B(soup2.text).encode('utf8').count(key)
    
    #資訊欄位
    for tm in soup1.select('tr'):
        #print tm
        #日期
        if u'予告開始日' in tm.text:
            #print 'ooooooooooooooooooooooooook',tm.td
            check=1
            cdata=tm.td.text
        elif u'販売日' in tm.text:
            cdata=tm.td.text
        #類型
        if u'作品形式'in tm.text:
            ctype='['+tm.td.text+']'
        #作者
        ##作者不一定有<a>，棄用的方法。
        #tmm=tm.select('a')#.get_text('//')
        tmm=tm.td
        tm=Q2B(tm.text).replace('\n','')
        #tm=Q2B(tm.get_text())
        if key.decode('utf8') in tm:
            
            #for tmmm in tmm:
                #cname=cname+tmmm.text+u'/'
            #print tmm,len(tmm)
            cname=Q2B(tmm.text).replace('\n','')
            bdate1=bdate1+tm+'_'
    bdate1=bdate1[:-1]
    #cname=Q2B(cname[:-1])
    #print 'cname:',len(cname),cname
    
    #內容欄位
    x=0
    if nnum2 > 0:
        tm=Q2B(soup2.text)
        while tm.find(key)>-1:
            #print 's2',tm[tm.rfind('\n',0,tm.find(key))+1:tm.find('\n',tm.find(key))]
            #key前後\n
            bdate2=bdate2+'-'+tm[tm.rfind(u'\n',0,tm.find(key))+1 : tm.find(u'\n',tm.find(key))].replace(u'\n','')+'\n'
            #print tm.find(key),tm.rfind('\n',0,tm.find(key)),tm.find('\n',tm.find(key))
            tm=tm[tm.find(key)+1:]
            x=x+1
            if x>5:
                break
    #\n佔用兩格
    bdate2=bdate2[:-2]
    #sou = BeautifulSoup(str(tr),"lxml")
    #'''
    
    if int(nnum1) > 0:
        check=5
        if u'マンガ' in ctype:
            check=2
            if u'同人' in ctype:
                check=4
            elif key.decode('utf8') != cname:
                check=3
                
    else:
        if u'マンガ' in ctype:
            check=6
        else:
            check=7
    
    
    
    '''
    1新
    2漫單
    3漫多
    4漫同人
    5它
    6漫
    7它
    1>4>3>2
    
    新
    合1:漫_漫多_同_它
    合2:漫_非
    #'''
    
    #print 'bfind'
    
    #類型_nnum1_bdate1_nnum2_bdate2_check
    #return cdata,ctype,nnum1,bdate1,nnum2,bdate2
    return ctype,nnum1,bdate1,nnum2,bdate2,check

##############################

#key='games'
#c1=bfind('http://www.dlsite.com/books/work/=/product_id/BJ011458.html')
c2=bfind('http://www.dlsite.com/books/work/=/product_id/BJ041839.html')
#c3=bfind('http://www.dlsite.com/books/work/=/product_id/BJ043185.html')
print '########catch:',len(c2)#,cfind[:]
#b1=c1[4]
b2=c2[4]
#b3=c3[4]

#print c1[4][:]+'_\n'+c2[4]+'_\n'+c3[4]+'_\n'
print b2+'_\n'
print type(b2),len(b2)
B2=b2.encode('utf8')
print B2[:]

########catch: 6
-『かたこい』hisasi
-モグダン/ごばん/ねこめたる/逢魔刻壱/水原優/hisasi/ぴょん吉/或十せねか/墓場_

<type 'unicode'> 60
-『かたこい』hisasi
-モグダン/ごばん/ねこめたる/逢魔刻壱/水原優/hisasi/ぴょん吉/或十せねか/墓場
